In [ ]:
# Установка необходимых пакетов
!pip install blocksnet ipykernel -q
!pip install folium matplotlib mapclassify
!pip install osmnx networkx ipykernel -q
!pip install mapclassify -q
!pip install iduedu
!pip install momepy

In [ ]:
# Импорт библиотек
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import momepy
from shapely.geometry import Point, Polygon, LineString
from blocksnet import BlocksGenerator, BlocksSplitter, AccessibilityProcessor

In [ ]:
# Загрузка данных
boundary = ('boundary.geojson')
territory= ('boundary.geojson')
roads= ('boundary.geojson')
roads_inside= ('boundary.geojson')
railways = ('boundary.geojson')
water = ('boundary.geojson')
buildings= ('boundary.geojson')


In [ ]:
# Загружаем данные
boundary = gpd.read_file(boundary)
if boundary.crs != "EPSG:3857":
      boundary = boundary.to_crs("EPSG:3857")

territory = gpd.read_file(territory)
if territory.crs != "EPSG:3857":
      territory = territory.to_crs("EPSG:3857")

roads = gpd.read_file(roads)
if roads.crs != "EPSG:3857":
      roads = roads.to_crs("EPSG:3857")


roads_inside = gpd.read_file(roads_inside)
if roads_inside.crs != "EPSG:3857":
      roads_inside = roads_inside.to_crs("EPSG:3857")


railways = gpd.read_file(railways)
if railways.crs != "EPSG:3857":
      railways = railways.to_crs("EPSG:3857")

water = gpd.read_file(water)
if water.crs != "EPSG:3857":
      water = water.to_crs("EPSG:3857")

buildings = gpd.read_file(buildings)
if buildings.crs != "EPSG:3857":
      buildings = buildings.to_crs("EPSG:3857")


In [ ]:
# Обработка дорог
roads = roads[roads.geom_type.isin(['LineString', 'MultiLineString'])]

GAP_TOLERANCE = 1

def _get_roads(roads):
    """Обработка и объединение дорожной сети"""
    merged = roads.unary_union
    if merged.geom_type == 'MultiLineString':
        roads = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=roads.crs)
    else:
        roads = gpd.GeoDataFrame(geometry=[merged], crs=roads.crs)
    
    roads = roads.explode(index_parts=False).reset_index(drop=True)
    roads.geometry = momepy.close_gaps(roads, GAP_TOLERANCE)
    roads = roads[roads.geom_type.isin(['LineString'])]
    return roads

roads = _get_roads(roads)

In [ ]:
# Генерация блоков
bg = BlocksGenerator(boundary, roads, railways)
blocks = bg.run()

In [ ]:
# Обработка зданий
buildings.geometry = buildings.representative_point()

In [ ]:
# Разделение блоков по зданиям
bs = BlocksSplitter(blocks, buildings)
splitted_blocks = bs.run()

In [ ]:
print(f"Количество блоков до и после разделения: {len(blocks)}, {len(splitted_blocks)}")

In [ ]:
blocks.plot(linewidth=0.1, figsize=(5,5)).set_axis_off()
splitted_blocks.plot(linewidth=0.1, figsize=(5,5)).set_axis_off()
splitted_blocks.to_file('blocks.geojson')

In [ ]:
# Анализ доступности
CRS = 32636
SPEED_M_MIN = 1000

def _roads_to_graph(roads):
    """Преобразование дорог в граф"""
    graph = momepy.gdf_to_nx(roads)
    graph.graph['crs'] = roads.crs.to_epsg()
    graph = nx.DiGraph(graph)
    
    for _, _, data in graph.edges(data=True):
        geometry = data['geometry']
        data['time_min'] = geometry.length / SPEED_M_MIN
        
    for n, data in graph.nodes(data=True):
        graph.nodes[n]['x'] = n[0]  # X координата
        graph.nodes[n]['y'] = n[1]  # Y координата

    return graph

roads_G = _roads_to_graph(roads)
AccessibilityProcessor._fix_graph(roads_G)

In [ ]:
# Расчет матрицы доступности
ap = AccessibilityProcessor(blocks)
acc_mx = ap.get_accessibility_matrix(roads_G)
acc_mx.head()  # вывод первых 5 строк матрицы доступности

In [ ]:
from blocksnet.models import City
from blocksnet import Accessibility, Connectivity

blocks['land_use'] = None

city = City(
    blocks=blocks,
    acc_mx=acc_mx
)

connectivity = Connectivity(city_model=city)
connectivity_result = connectivity.calculate()
connectivity_result

In [ ]:
connectivity_result.explore()

In [ ]:
from blocksnet.models import City
blocks['land_use'] = None
city = City(
    blocks=blocks,
    acc_mx=acc_mx
)

Доступность

In [ ]:
from blocksnet import Accessibility, Connectivity
accessibility = Accessibility(city_model=city)
block = city[159] # квартал от которого будем считать доступность
result = accessibility.calculate(block)
#Accessibility.plot(result, linewidth=0.9, figsize=(30,15))


In [ ]:
block

In [ ]:
result.plot(column='accessibility_from', legend=True, figsize=(10,10), cmap='RdYlGn_r').set_axis_off()
result.plot(column='accessibility_to', legend=True, figsize=(5,5), cmap='RdYlGn_r').set_axis_off()

In [ ]:
# Пример DataFrame с кварталами и значениями доступности
data = result

# Создаем DataFrame
df = pd.DataFrame(data)

# Рассчитываем среднее и медиану для каждого атрибута
mean_to = df['accessibility_to'].mean()
median_to = df['accessibility_to'].median()

mean_from = df['accessibility_from'].mean()
median_from = df['accessibility_from'].median()

# Добавляем столбцы с нормализованными значениями
df['accessibility_to_normalized'] = (df['accessibility_to'] - df['accessibility_to'].min()) / (df['accessibility_to'].max() - df['accessibility_to'].min())
df['accessibility_from_normalized'] = (df['accessibility_from'] - df['accessibility_from'].min()) / (df['accessibility_from'].max() - df['accessibility_from'].min())

# Выводим результаты
print(f"Среднее значение доступности 'to': {mean_to:.2f}")
print(f"Медиана доступности 'to': {median_to:.2f}")
print(f"Среднее значение доступности 'from': {mean_from:.2f}")
print(f"Медиана доступности 'from': {median_from:.2f}")

Связанность

In [ ]:
connectivity = Connectivity(city_model=city)
connectivity_result = connectivity.calculate()
Connectivity.plot(connectivity_result, linewidth=0.9, figsize=(10,10))

In [ ]:
dff = pd.DataFrame(connectivity_result)
stats = dff['connectivity'].agg(['mean', 'median'])
mean_to = dff['connectivity'].mean()
median_to = dff['connectivity'].median()
# Добавляем столбцы с нормализованными значениями
dff['connectivity_to_normalized'] = (dff['connectivity'] - dff['connectivity'].min()) / (dff['connectivity'].max() - dff['connectivity'].min())

# Выводим результаты
print(f"Среднее значение связанности: {mean_to:.2f}")
print(f"Медиана связанности: {median_to:.2f}")

Площадь кварталов

In [ ]:
blocks_area = gpd.clip(blocks, territory)

In [ ]:
if blocks_area.crs != "EPSG:3857":
      blocks_area = blocks_area.to_crs("EPSG:3857")

In [ ]:
def calculate_average_block_size(blocks_area):

    blocks_area['block_size'] = blocks_area.geometry.area
    filtered_blocks = blocks_area[blocks_area['block_size'] >= 1000]
    if filtered_blocks.empty:
        raise ValueError("Нет кварталов с площадью >= 1000 м² для расчета.")
    return filtered_blocks['block_size'].mean()

try:
    average_block_size = calculate_average_block_size(blocks_area)
    average_block_size_ha = average_block_size / 10_000
    print(f"Средняя площадь квартала: {average_block_size:.2f} м² ({average_block_size_ha:.2f} га)")
except ValueError as e:
    print(e)

Плотность УДС

In [ ]:
def street_density(roads_inside, territory):
    """Плотность улиц (км/км²)"""
    roads_inside = roads_inside.to_crs(territory.crs) if roads_inside.crs != territory.crs else roads_inside
    return (roads_inside.geometry.length.sum() / 1000) / (territory.geometry.area.sum() / 1e6)

# Пример вызова
print(f"Плотность удс: {street_density(roads_inside, territory):.2f} км/км²")